# Hetero NN 单机版快速开始

在该版本中， Hetero-NN同Homo-NN一样进行了升级，Hetero-NN与Homo-NN一起共享dataset, model_zoo模块，因此，在此版本中，Hetero-NN也可以允许模型的
高度定制化，以及数据集的定制化，我们会在Hetero-NN相关的下一章介绍定制化内容

同时Hetero-NN还升级了部分接口，如Interactive-layer的接口，使得使用逻辑更为清晰

本章中，给出一个基础的二分类任务Hetero-NN任务， 使用方法与的其他FATE算法一致：使用FATE自带的reader, transformer接口进行表格数据输入，
数据输入到算法组件中，组件使用定义的模型，优化器和loss函数进行训练，该版本的使用方式与FATE-1.9大体一致

在下一章，我们会介绍如何使用自定义模型与自定义数据集，与本章例子会有不同

## 上传csv数据到FATE

In [5]:
from pipeline.backend.pipeline import PipeLine  # pipeline类

dense_data_guest = {"name": "breast_hetero_guest", "namespace": f"experiment"}
dense_data_host = {"name": "breast_hetero_host", "namespace": f"experiment"}

guest= 9999
host = 10000

pipeline_upload = PipeLine().set_initiator(role='guest', party_id=guest).set_roles(guest=guest, host=host)

partition = 4

# 上传一份数据
pipeline_upload.add_upload_data(file="./examples/data/breast_hetero_guest.csv",
                                table_name=dense_data_guest["name"],             # table name
                                namespace=dense_data_guest["namespace"],         # namespace
                                head=1, partition=partition)               # data info

pipeline_upload.add_upload_data(file="./examples/data/breast_hetero_host.csv",
                                table_name=dense_data_host["name"],
                                namespace=dense_data_host["namespace"],
                                head=1, partition=partition)      # data info

pipeline_upload.upload(drop=1)

 UPLOADING:||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||100.00%


2022-11-03 19:57:25.382 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202211031957250954390

2022-11-03 19:57:25.393 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
2022-11-03 19:57:26.410 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
m2022-11-03 19:57:27.431 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2022-11-03 19:57:27.432 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:02
2022-11-03 19:57:28.455 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:03
2022-11-03 19:57:29.484 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:04
2022-11-03 19:57:30.510 | INFO     | pipel

 UPLOADING:||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||100.00%


2022-11-03 19:57:31.968 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202211031957315418150

2022-11-03 19:57:31.986 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
2022-11-03 19:57:32.999 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
2022-11-03 19:57:34.013 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:02
m2022-11-03 19:57:35.033 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2022-11-03 19:57:35.034 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:03
2022-11-03 19:57:36.055 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:04
2022-11-03 19:57:37.084 | INFO     | pipeline.uti

## 编写Pipeline脚本并执行

In [2]:
import torch as t
from torch import nn
from pipeline import PipeLine
from pipeline import fate_torch_hook
from pipeline.component import HeteroNN, Reader, DataTransform, Intersection
from pipeline.component.hetero_nn import DatasetParam
from pipeline.interface import Data, Model

fate_torch_hook(t)

guest = 9999
host = 10000
pipeline = PipeLine().set_initiator(role='guest', party_id=guest).set_roles(guest=guest, host=host)

guest_train_data = {"name": "breast_hetero_guest", "namespace": "experiment"}
host_train_data = {"name": "breast_hetero_host", "namespace": "experiment"}

pipeline = PipeLine().set_initiator(role='guest', party_id=guest).set_roles(guest=guest, host=host)

reader_0 = Reader(name="reader_0")
reader_0.get_party_instance(role='guest', party_id=guest).component_param(table=guest_train_data)
reader_0.get_party_instance(role='host', party_id=host).component_param(table=host_train_data)

data_transform_0 = DataTransform(name="data_transform_0")
data_transform_0.get_party_instance(role='guest', party_id=guest).component_param(with_label=True)
data_transform_0.get_party_instance(role='host', party_id=host).component_param(with_label=False)

intersection_0 = Intersection(name="intersection_0")

### hetero nn 组件

In [3]:
hetero_nn_0 = HeteroNN(name="hetero_nn_0", epochs=20,
                       interactive_layer_lr=0.01, batch_size=-1, validation_freqs=1, task_type='classification', seed=114514)
guest_nn_0 = hetero_nn_0.get_party_instance(role='guest', party_id=guest)
host_nn_0 = hetero_nn_0.get_party_instance(role='host', party_id=host)

### 定义模型 往guest&host 添加模型

In [4]:
# 定义模型
guest_bottom = t.nn.Sequential(
    nn.Linear(10, 2),
    nn.ReLU()
)

guest_top = t.nn.Sequential(
    nn.Linear(2, 1),
    nn.Sigmoid()
)

host_bottom = t.nn.Sequential(
    nn.Linear(20, 2),
    nn.ReLU()
)

# fate_torch_hook后，nn模块可以使用InteractiveLayer, 你可以查看Interactive layer的结构
interactive_layer = t.nn.InteractiveLayer(out_dim=2, guest_dim=2, host_dim=2, host_num=1)

print(interactive_layer)

guest_nn_0.add_top_model(guest_top)
guest_nn_0.add_bottom_model(guest_bottom)
host_nn_0.add_bottom_model(host_bottom)

optimizer = t.optim.Adam(lr=0.01) # 注意！fate_torch_hook后，优化器可以不用parameter参数
loss = t.nn.BCELoss()

hetero_nn_0.set_interactve_layer(interactive_layer)
hetero_nn_0.add_dataset(DatasetParam(dataset_name='table'))
hetero_nn_0.compile(optimizer=optimizer, loss=loss)

InteractiveLayer(
  (activation): ReLU()
  (guest_model): Linear(in_features=2, out_features=2, bias=True)
  (host_model): ModuleList(
    (0): Linear(in_features=2, out_features=2, bias=True)
  )
  (act_seq): Sequential(
    (0): ReLU()
  )
)


In [5]:
pipeline.add_component(reader_0)
pipeline.add_component(data_transform_0, data=Data(data=reader_0.output.data))
pipeline.add_component(intersection_0, data=Data(data=data_transform_0.output.data))
pipeline.add_component(hetero_nn_0, data=Data(train_data=intersection_0.output.data))
pipeline.compile()

In [6]:
pipeline.fit()

2022-11-04 14:28:44.655 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202211041428441512400

2022-11-04 14:28:44.666 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
2022-11-04 14:28:45.677 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
m2022-11-04 14:28:46.706 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2022-11-04 14:28:46.707 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:02
2022-11-04 14:28:47.734 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:03
2022-11-04 14:28:48.770 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:04
2022-11-04 14:28:49.806 | INFO     | pipel

2022-11-04 14:29:23.196 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:00:38
2022-11-04 14:29:24.228 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:00:39
2022-11-04 14:29:25.257 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:00:40
2022-11-04 14:29:26.292 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:00:41
2022-11-04 14:29:27.327 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:00:42
2022-11-04 14:29:28.364 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:00:43
2022-11-04 14:29:29.394 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:1

2022-11-04 14:30:03.721 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:19
2022-11-04 14:30:04.756 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:20
2022-11-04 14:30:05.784 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:21
2022-11-04 14:30:06.817 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:22
2022-11-04 14:30:07.854 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:23
2022-11-04 14:30:08.898 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:24
2022-11-04 14:30:09.930 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:1

2022-11-04 14:30:44.286 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:59
2022-11-04 14:30:45.316 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:02:00
2022-11-04 14:30:46.357 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:02:01
2022-11-04 14:30:47.392 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:02:02
2022-11-04 14:30:48.429 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:02:03
2022-11-04 14:30:49.468 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:02:04
2022-11-04 14:30:50.496 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:1